# Projet de modélisation d'un mix énergétique issu 100% d'ENR

## III. Simulations du modèle sur l'optimisation du coût du mix énergétique

### Imports

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

# Imports pour pyomo
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from ipywidgets import interactive, interact_manual
import ipywidgets as widgets

from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvas

## 1. Utils

In [2]:
col = {"offshore" : '#1f4e79', "onshore" : '#548235' , "pv" : "#ffc000", "phs" : "#bdd7ee", "battery" : "#7030a0", "biogas" :"#843c0c", "methanation": "#7f6000", "gas": "grey", "lake": "#4169E1", "river" : "#09F1D3"}
tec = ["offshore","onshore","pv","river","lake","biogas","phs","battery","methanation"]
stor = ["phs","battery","methanation"]

In [3]:
def affichage(simu):
    print(" --- Q ---")
    for i in simu.model.Q:
        print(i," : ",pyo.value(simu.model.Q[i]))   
    print(" --- S ---")   
    for i in simu.model.S:
        print(i," : ",pyo.value(simu.model.S[i]))
    print(" --- volume ---")
    for i in simu.model.volume:
        print(i," : ",pyo.value(simu.model.volume[i]))
    print(" --- cost --- ")
    print(simu.cost())

In [4]:
# Optimal values found in summary with demand of 2016.txt
Volume_str_opti = pd.Series([180 ,74.14, 12499.09], index = stor)
S_opti = pd.Series([9.3,20.08, 7.66], index = stor)
Q_tec_opti = pd.Series([12.64, 79.73, 121.98, 7.5, 13 ,32.93, 9.3, 20.08, 32.93], index = tec)
Cost_opti = 21.84 

## 2. Simulation du modèle EOLES

### 2.1. Chois de Q, S et VOLUME

In [5]:
phs_bounds = {"Q_lower":5.2,"Q_upper": 9.3, "S_lower":4.2, "S_upper":9.3, "Volume_lower":80.16, "Volume_upper":180}
vre_upper_bounds = {"onshore":120, "offshore": 20, "pv": 218}

In [6]:
V = widgets.Text(value='Volume de stockage (GWh)', disabled=True)
Volume_phs = widgets.FloatSlider(min=phs_bounds["Volume_lower"], max=phs_bounds["Volume_upper"], step=0.01, description='Volume phs',value = Volume_str_opti["phs"])
Volume_methanation = widgets.FloatSlider(min=0, max=Volume_str_opti["methanation"]*5, step=0.01, description='Volume methanation',value = Volume_str_opti["methanation"])
Volume_battery =  widgets.FloatSlider(min=0, max=Volume_str_opti["battery"]*5, step=0.01, description='Volume battery',value = Volume_str_opti["battery"])  

Q = widgets.Text(value='Capcité installée (GW)', disabled=True)
Q_offshore= widgets.FloatSlider(min=0, max=vre_upper_bounds["offshore"], step=0.01, description='Q offshore',value = Q_tec_opti["offshore"])
Q_onshore= widgets.FloatSlider(min=0, max=vre_upper_bounds["onshore"], step=0.01, description='Q onshore',value = Q_tec_opti["onshore"])
Q_pv= widgets.FloatSlider(min=0, max=vre_upper_bounds["pv"], step=0.1, description='Q pv',value = Q_tec_opti["pv"])
Q_river= widgets.FloatSlider(min=0, max=Q_tec_opti["river"], step=0.01, description='Q river',value = Q_tec_opti["river"])
Q_lake= widgets.FloatSlider(min=12.855, max=13, step=0.01, description='Q lake',value = Q_tec_opti["lake"])
Q_biogas= widgets.FloatSlider(min=0, max=Q_tec_opti["biogas"]*5, step=0.01, description='Q biogas',value = Q_tec_opti["biogas"])
Q_phs= widgets.FloatSlider(min=phs_bounds["Q_lower"], max=phs_bounds["Q_upper"], step=0.01, description='Q phs',value = Q_tec_opti["phs"])
Q_battery= widgets.FloatSlider(min=0, max=Q_tec_opti["battery"]*5, step=0.01, description='Q battery',value = Q_tec_opti["battery"])
Q_methanation= widgets.FloatSlider(min=0, max=Q_tec_opti["methanation"]*5, step=0.01, description='Q methanation',value = Q_tec_opti["methanation"])

S = widgets.Text(value='Capacité de charge (GW)', disabled=True)
S_phs= widgets.FloatSlider(min=phs_bounds["S_lower"], max=phs_bounds["S_upper"], step=0.01, description='S phs',value = S_opti["phs"])
S_methanation = widgets.FloatSlider(min=0, max=S_opti["methanation"]*5, step=0.01, description='S methanation',value = S_opti["methanation"])
warn = widgets.Textarea(value='      Attention ! \n      Volume > Q  et  Volume > S', disabled=True)

widgets.HBox([widgets.VBox([Q, Q_offshore, Q_onshore, Q_pv, Q_river, Q_lake,Q_biogas ,Q_phs,Q_battery,Q_methanation]),
widgets.VBox([V,Volume_phs,Volume_battery,Volume_methanation, warn]), widgets.VBox([S,S_methanation, S_phs])])

In [7]:
#Valeurs modifiées par les curseurs :
Q_tec = [Q_offshore.value, Q_onshore.value, Q_pv.value, Q_river.value, Q_lake.value,Q_biogas.value ,Q_phs.value,Q_battery.value,Q_methanation.value]
Volume_str = [Volume_phs.value,Volume_battery.value,Volume_methanation.value]
S =  [Q_phs.value,Q_battery.value,S_methanation.value]

# Capacity in GW Q
Q_tec = pd.Series( Q_tec, index = tec)

# Energy volume of storage technology in GWh VOLUME
Volume_str = pd.Series(Volume_str, index = stor)

#Storage charging power S
S = pd.Series(S, index = stor)

print("Fixed values : ")
print("--------- Q ---------")
print(Q_tec)
print("-------- VOLUME -------")
print(Volume_str)
print("--------- S ---------")
print(S)

Fixed values : 
--------- Q ---------
offshore        12.64
onshore         79.73
pv             121.98
river            7.50
lake            13.00
biogas          32.93
phs              9.30
battery         20.08
methanation     32.93
dtype: float64
-------- VOLUME -------
phs              180.00
battery           74.14
methanation    12499.09
dtype: float64
--------- S ---------
phs             9.30
battery        20.08
methanation     7.66
dtype: float64


### <span style="color:blue">-------------------------------------------------------------------------------------------------------------------------------------------------------</span>
### <span style="color:blue">A exécuter si on n'a pas le dossier Simulations_opti</span>

### 2.2 Simulation avec les paramètres optimaux de l'année 2006

In [8]:
from opti_model import opti_model

#### Demande de RTE

In [9]:
rte = pd.read_csv("inputs/demand2050_rte.csv")
rte

,0,77.1753112996997
0,1,73.642027
1,2,68.335901
2,3,66.023141
3,4,63.549687
4,5,61.352233
...,...,...
8730,8731,79.932693
8731,8732,82.225989
8732,8733,80.628026
8733,8734,77.000773


In [10]:
ademe = pd.read_csv("inputs/demand2050_negawatt.csv")
ademe

,0,27.698363
0,1,26.283084
1,2,25.100329
2,3,23.808746
3,4,22.558374
4,5,22.404221
...,...,...
8754,8755,37.314774
8755,8756,38.171687
8756,8757,31.211406
8757,8758,29.594038


#### Demande de l'ADEME

In [15]:
#Run the simulation
simu1 = opti_model(Q_tec, Volume_str, S)
simu1.load_param("inputs/", "vre_profiles2006.csv", "demand2050_ademe.csv")
simu1.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vre_profiles2006.csv
demand2050_ademe.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [16]:
#Write results
res1 = simu1.write_results('simu_opti_ademe', "vre_profiles2006.csv", "demand2050_ademe.csv")

Ecriture des résultats ...


C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


#### Demande de Négawatt

In [17]:
#Run the simulation
simu2 = opti_model(Q_tec, Volume_str, S)
simu2.load_param("inputs/", "vre_profiles2006.csv", "demand2050_negawatt.csv")
simu2.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vre_profiles2006.csv
demand2050_negawatt.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [18]:
#Write results
res2 = simu2.write_results('simu_opti_negawatt', "vre_profiles2006.csv", "demand2050_negawatt.csv")

Ecriture des résultats ...


C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [19]:
#Run the simulation
simu3 = opti_model(Q_tec, Volume_str, S)
simu3.load_param("inputs/", "vre_profiles2006.csv", "demand2006.csv")
simu3.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vre_profiles2006.csv
demand2006.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Simulation du modèle faite avec succès ! 


In [20]:
#Write results
res3 = simu3.write_results('simu_opti_d2006', "vre_profiles2006.csv", "demand2006.csv")

Ecriture des résultats ...


C:\Users\teaca\OneDrive - insa-toulouse.fr\Documents\INSA\4A\4A.Stage\4A.Stage.Codes_ENR\modele.py:286: RuntimeWarning: divide by zero encountered in double_scalars
  c_mwh_produced = c_tot/sumgene
C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [21]:
#Run the simulation
simu4 = opti_model(Q_tec, Volume_str, S)
simu4.load_param("inputs/", "vrenew_typical_days_new_trend_2006.csv", "demand2006.csv")
simu4.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vrenew_typical_days_new_trend_2006.csv
demand2006.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Simulation du modèle faite avec succès ! 


In [22]:
#Write results
res4 = simu4.write_results('simu_opti_td2006_d2006', "vrenew_typical_days_new_trend_2006.csv", "demand2006.csv")

Ecriture des résultats ...


C:\Users\teaca\OneDrive - insa-toulouse.fr\Documents\INSA\4A\4A.Stage\4A.Stage.Codes_ENR\modele.py:286: RuntimeWarning: divide by zero encountered in double_scalars
  c_mwh_produced = c_tot/sumgene
C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [23]:
#Run the simulation
simu5 = opti_model(Q_tec, Volume_str, S)
simu5.load_param("inputs/", "vrenew_typical_days_new_trend_2006.csv", "demand2050_rte.csv")
simu5.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vrenew_typical_days_new_trend_2006.csv
demand2050_rte.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Simulation du modèle faite avec succès ! 


In [24]:
#Write results
res5 = simu5.write_results('simu_opti_td2006_rte', "vrenew_typical_days_new_trend_2006.csv", "demand2050_rte.csv")

Ecriture des résultats ...


C:\Users\teaca\OneDrive - insa-toulouse.fr\Documents\INSA\4A\4A.Stage\4A.Stage.Codes_ENR\modele.py:286: RuntimeWarning: divide by zero encountered in double_scalars
  c_mwh_produced = c_tot/sumgene
C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [25]:
#Run the simulation
simu6 = opti_model(Q_tec, Volume_str, S)
simu6.load_param("inputs/", "vrenew_typical_days_new_trend_2006.csv", "demand2050_ademe.csv")
simu6.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vrenew_typical_days_new_trend_2006.csv
demand2050_ademe.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [26]:
#Write results
res6 = simu6.write_results('simu_opti_td2006_ademe', "vrenew_typical_days_new_trend_2006.csv", "demand2050_ademe.csv")

Ecriture des résultats ...


C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [27]:
#Run the simulation
simu7 = opti_model(Q_tec, Volume_str, S)
simu7.load_param("inputs/", "vrenew_typical_days_new_trend_2006.csv", "demand2050_negawatt.csv")
simu7.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vrenew_typical_days_new_trend_2006.csv
demand2050_negawatt.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
Simulation du modèle faite avec succès ! 


In [28]:
#Write results
res7 = simu7.write_results('simu_opti_td2006_negawatt', "vrenew_typical_days_new_trend_2006.csv", "demand2050_negawatt.csv")

Ecriture des résultats ...


C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


In [15]:
#Run the simulation
simu8 = opti_model(Q_tec, Volume_str, S)
simu8.load_param("inputs/", "vrenew_typical_days_new_trend_2014.csv", "demand2050_rte.csv")
simu8.run()

Technologies utilisées : ['offshore', 'onshore', 'pv', 'river', 'lake', 'biogas', 'phs', 'battery', 'methanation']
Chargement des paramètres ...
vrenew_typical_days_new_trend_2014.csv
demand2050_rte.csv
Initialisation des sets...
Définition des variables à optimiser ...
Ajout des contraintes ...
Optimisation ...
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Simulation du modèle faite avec succès ! 


In [17]:
#Write results
res8 = simu8.write_results('simu_opti_td2014_rte', "vrenew_typical_days_new_trend_2014.csv", "demand2050_rte.csv")

Ecriture des résultats ...


C:\Users\teaca\OneDrive - insa-toulouse.fr\Documents\INSA\4A\4A.Stage\4A.Stage.Codes_ENR\opti_model.py:286: RuntimeWarning: divide by zero encountered in double_scalars
  c_mwh_produced = c_tot/sumgene
C:\Users\teaca\anaconda3\lib\site-packages\pandas\core\internals\construction.py:540: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


### <span style="color:blue">-------------------------------------------------------------------------------------------------------------------------------------------------------</span>

### 2.3 Analyse des résultats

In [3]:
from opti_model import show_simu

##### Composition du mix énergétique en fonction de la demande : ADEME ou NégaWatt

In [4]:
noms_simu = ["ademe", "negawatt","rte"]
# Dashboard
interactive_plot = interact_manual(show_simu, week=widgets.IntText(value = 1), name = noms_simu)
interactive_plot

interactive(children=(IntText(value=1, description='week'), Dropdown(description='name', options=('ademe', 'ne…

<function opti_model.show_simu(week, name, opti=True)>

L'été on stocke l'énergie solaire en surplus la journée pour alimenter le réseau la nuit par PHS et batteries : court terme. On utilise également l'électricité générée par les barages.  
L'hiver, l'énergie solaire est moins présente, on génère de l'électricité avec l'éolien et on utilise massivement le stockage par méthanation : long terme. 